#Random Forest Modle
###ECS171 Group 19
###Zherui Xie

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

class Reader:
    def __init__(self, path, mode='f'):
        
        self.df = self.wrapper(path,mode)
    
    # read arff file
    def readarff(self, filename,collect=True): #collect if you need to collect attribute names
        with open(filename) as f:
            content = f.read().splitlines()
        data = False
        metalist = [] # storets metadata as list of rows
        datalist = [] # store data as list of rows
        
        # read data line-by-line
        for line in content:
            if data == True:
                line = line.split(",")
                datalist.append(line)
            elif line == "@data":
                data = True # read lines before "@data" as metadata and after as data
            else:
              # clean up metadata header
              if collect:
                line = line.replace(' "', ".")
                line = line.replace('" ', ".")
                line = line.replace(" ","")
                line = line.split(".")
                if len(line)==3: #ignore first two lines of file
                    line = line[1:3] #remove repetitive "@attribute"
                    metalist.append(line)
        
        # create dataframes from lists of rows
        if not collect:
            dataframe = pd.DataFrame(datalist,dtype=float)
            return dataframe
        else:
            dataframe = pd.DataFrame(datalist,dtype=float)
            metaframe = pd.DataFrame(metalist,columns=["attribute","description"])
            attributes = metaframe["attribute"].rename("SAMPLE")
            return dataframe, attributes
    
    def readdirectory(self, path,quiet=False): # make sure path ends in a slash
        alldata = []
        count = 0
        for filename in os.listdir(path):
            if filename.endswith(".arff"):
                if count == 0: #only collect attributes once
                    if not quiet:
                          print("processing "+filename+"; collecting attribute names")
                    dataframe, attributes = self.readarff(path+filename)
                    alldata.append(dataframe)
                else:
                    if not quiet:
                        print("processing "+filename)
                    dataframe = self.readarff(path+filename,collect=False)
                    alldata.append(dataframe)
                count += 1
                continue
            else:
                continue
        if not quiet:
            print("Concatenating data")
        alldata = pd.concat(alldata).reset_index(drop=True) #reset indices so it is continuous
        alldata.columns = attributes #assign column names
        return alldata
    
    def wrapper(self, path, mode='f'):
        if mode == 'f':
            try:
                df =  self.readarff(path, collect = True)
                return df
            except:
                print("make sure you inputted the correct arff FILE path")
        elif mode == 'd':
            try:
                df =  self.readdirectory(path, quiet=True)
                return df
            except:
                print("make sure you inputted the correct arff DIRECTORY path, ending with a slash")
        else:
            print("mode must either be 'f' or 'd'")
            return 0

In [4]:
phone_accel=Reader("/content/drive/My Drive/Colab Notebooks/datasets/phone_accel/",mode='d').df
phone_accel

SAMPLE,ACTIVITY,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,XAVG,YAVG,ZAVG,XPEAK,YPEAK,ZPEAK,XABSOLDEV,YABSOLDEV,ZABSOLDEV,...,XMFCC7,XMFCC8,XMFCC9,XMFCC10,XMFCC11,XMFCC12,YMFCC0,YMFCC1,YMFCC2,YMFCC3,YMFCC4,YMFCC5,YMFCC6,YMFCC7,YMFCC8,YMFCC9,YMFCC10,YMFCC11,YMFCC12,ZMFCC0,ZMFCC1,ZMFCC2,ZMFCC3,ZMFCC4,ZMFCC5,ZMFCC6,ZMFCC7,ZMFCC8,ZMFCC9,ZMFCC10,ZMFCC11,ZMFCC12,XYCOS,XZCOS,YZCOS,XYCOR,XZCOR,YZCOR,RESULTANT,class
0,A,0.005,0.045,0.165,0.380,0.240,0.070,0.030,0.020,0.020,0.025,0.010,0.025,0.025,0.205,0.135,0.265,0.165,0.105,0.015,0.050,0.135,0.235,0.410,0.070,0.055,0.055,0.035,0.005,0.000,0.0,5.07132,8.28927,1.034070,35.4717,34.0000,31.2698,2.750970,3.530930,2.742550,...,0.649454,0.644151,0.638159,0.631483,0.624132,0.616111,0.493183,0.697093,0.695973,0.694108,0.691499,0.688150,0.684065,0.679246,0.673701,0.667434,0.660452,0.652763,0.644375,0.457278,0.646343,0.645305,0.643575,0.641157,0.638052,0.634263,0.629796,0.624654,0.618843,0.612370,0.605240,0.597463,0.799349,0.046818,0.258132,0.380278,-0.253138,0.077059,11.08710,1601.0
1,A,0.010,0.060,0.165,0.405,0.215,0.065,0.025,0.020,0.025,0.010,0.005,0.010,0.080,0.145,0.110,0.355,0.155,0.060,0.030,0.050,0.155,0.250,0.360,0.105,0.055,0.045,0.015,0.015,0.000,0.0,4.73710,8.43164,0.849532,36.3462,34.7273,32.5424,2.651210,3.337670,2.839390,...,0.651608,0.646288,0.640276,0.633579,0.626202,0.618156,0.505783,0.714902,0.713754,0.711841,0.709166,0.705732,0.701541,0.696600,0.690913,0.684485,0.677325,0.669440,0.660837,0.465067,0.657352,0.656296,0.654537,0.652077,0.648919,0.645067,0.640523,0.635293,0.629384,0.622800,0.615549,0.607639,0.781673,0.028650,0.248155,0.317421,-0.216656,0.141743,10.94270,1601.0
2,A,0.000,0.070,0.195,0.395,0.190,0.065,0.050,0.015,0.010,0.010,0.010,0.015,0.130,0.090,0.085,0.330,0.175,0.090,0.015,0.060,0.140,0.325,0.300,0.105,0.045,0.035,0.025,0.020,0.005,0.0,4.61791,8.47854,0.977499,35.2727,39.7959,32.6667,2.611000,3.654060,2.827690,...,0.634334,0.629155,0.623302,0.616782,0.609601,0.601768,0.509309,0.719885,0.718729,0.716803,0.714109,0.710651,0.706431,0.701455,0.695728,0.689256,0.682046,0.674106,0.665444,0.453175,0.640543,0.639514,0.637800,0.635403,0.632326,0.628572,0.624144,0.619048,0.613290,0.606874,0.599809,0.592102,0.805913,0.024328,0.309101,0.403558,-0.267118,0.217244,11.01720,1601.0
3,A,0.005,0.055,0.220,0.410,0.165,0.055,0.055,0.030,0.000,0.005,0.005,0.015,0.135,0.055,0.125,0.340,0.150,0.110,0.015,0.050,0.120,0.300,0.335,0.110,0.030,0.065,0.015,0.015,0.010,0.0,4.37941,8.53652,1.089330,40.4167,36.5385,33.9655,2.558040,3.469900,2.889170,...,0.615700,0.610673,0.604992,0.598663,0.591694,0.584090,0.491452,0.694647,0.693531,0.691672,0.689073,0.685736,0.681664,0.676863,0.671337,0.665091,0.658134,0.650472,0.642114,0.452133,0.639071,0.638044,0.636334,0.633943,0.630873,0.627127,0.622710,0.617626,0.611880,0.605479,0.598430,0.590741,0.777549,0.040502,0.332777,0.306777,-0.260409,0.238834,10.93550,1601.0
4,A,0.000,0.050,0.230,0.415,0.165,0.085,0.035,0.015,0.000,0.005,0.010,0.025,0.125,0.085,0.060,0.310,0.195,0.095,0.040,0.055,0.155,0.330,0.275,0.135,0.025,0.025,0.040,0.015,0.000,0.0,4.20028,8.69639,0.850603,36.8000,37.0588,31.3115,2.353640,3.795790,2.734410,...,0.606007,0.601059,0.595468,0.589239,0.582379,0.574895,0.512041,0.723747,0.722584,0.720648,0.717940,0.714463,0.710221,0.705218,0.699461,0.692954,0.685705,0.677722,0.669013,0.444736,0.628615,0.627605,0.625923,0.623571,0.620551,0.616867,0.612522,0.607521,0.601869,0.595573,0.588640,0.581076,0.810781,0.093274,0.327170,0.396817,-0.137979,0.282769,10.89910,1601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697,S,0.000,0.000,0.000,0.010,0.990,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.105,0.895,0.000,